<a href="https://colab.research.google.com/github/j03m/lstm-price-predictor/blob/main/Coin_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IMPORT DATASETS AND LIBRARIES


In [31]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [32]:
import pandas as pd
import plotly.express as px
from copy import copy
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, confusion_matrix, classification_report, accuracy_score, f1_score
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
import requests
from requests.exceptions import HTTPError
import json as js
from datetime import datetime, timedelta 
import time
from os.path import exists
from decimal import *
from sklearn.model_selection import RandomizedSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from sklearn.model_selection import KFold, ParameterGrid
from keras.layers import Input, LSTM, Attention, Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error


pd.set_option('display.float_format', lambda x: '%.5f' % x)

#Library

In [62]:
# Function to plot interactive plots using Plotly Express
sc = MinMaxScaler()
num_features = 3
coin_base = False
ku_coin = True
COINBASE_REST_API = 'https://api.pro.coinbase.com' 
COINBASE_PRODUCTS = COINBASE_REST_API+'/products'
KUCOIN_REST_API = "https://api.kucoin.com"
KUCOIN_PRODUCTS = KUCOIN_REST_API+ "/api/v1/market/allTickers"
KUCOIN_CANDLES = KUCOIN_REST_API+ "/api/v1/market/candles"

data_path = '/content/drive/My Drive/output.csv'
model_path = "/content/drive/My Drive"

def interactive_plot(df, title):
  fig = px.line(title = title)
  for i in df.columns[1:]:
    fig.add_scatter(x = df['Date'], y = df[i], name = i)
  fig.show()

def get_single_stock(price_df, vol_df, name):
    return pd.DataFrame({'Date': price_df['Date'], 'Close': price_df[name], 'Volume': vol_df[name]})

def scale_data(data):
  # Scale the data
  scaled_data = sc.fit_transform(data)
  return scaled_data

def sort_date(pric_df):
  pric_df = pric_df.sort_values(by = ['Date'])
  return pric_df

def append_price_dif(df):
  df['Target'] = df['Close'].shift(-1)
  df['Diff'] = df['Target'] - df['Close']
  df = df[:-1]
  return df

def append_price_dif_(df):
  df['Target'] = df['Close'].shift(-1)
  df['Diff'] = df['Target'] - df['Close']
  return df

def append_15d_slope(df):
  df['15Close'] = df['Close'].shift(15)
  df['15Date'] = df['Date'].shift(15)
  df['Trend'] = (df['Close'] - df['15Close']) / 15
  df = df[15:]
  return df

def show_plot(data, title):
  plt.figure(figsize = (13, 5))
  plt.plot(data, linewidth = 3)
  plt.title(title)
  plt.grid()

def build_model(features, outcomes):
  # Create the model
  inputs = keras.layers.Input(shape=(features,outcomes))
  x = keras.layers.LSTM(150, return_sequences= True)(inputs)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150, return_sequences=True)(x)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150)(x)
  outputs = keras.layers.Dense(1, activation='linear')(x)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer='adam', loss="mse")
  return model

def build_attention_model(features, outcomes):
  # Create the model
  inputs = keras.layers.Input(shape=(features,outcomes))
  x = keras.layers.LSTM(150, return_sequences= True)(inputs)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150, return_sequences=True)(x)
  x = keras.layers.Dropout(0.3)(x)
  x = keras.layers.LSTM(150)(x)
  attention_layer = Attention()([x, x])
  outputs = keras.layers.Dense(1, activation='linear')(x)  
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer='adam', loss="mse")
  return model

def connect(url, params):   
  response = requests.get(url,params)
  response.raise_for_status()
  return response

def coinbase_json_to_df(delta, product, granularity='86400'):
  start_date = (datetime.today() - timedelta(seconds=delta*int(granularity))).isoformat()
  end_date = datetime.now().isoformat()
  # Please refer to the coinbase documentation on the expected parameters
  params = {'start':start_date, 'end':end_date, 'granularity':granularity}
  response = connect(COINBASE_PRODUCTS+'/' + product + '/candles', params)
  response_text = response.text
  df_history = pd.read_json(response_text)
  # Add column names in line with the Coinbase Pro documentation
  df_history.columns = ['time','low','high','open','close','volume']
  df_history['time'] = [datetime.fromtimestamp(x) for x in df_history['time']]
  return df_history

def ku_coin_json_to_df(delta, product, granularity='86400'):
  granularity = int(granularity)
  start_date = (datetime.today() - timedelta(seconds=delta*granularity))
  end_date = datetime.now()

  # Please refer to the kucoin documentation on the expected parameters
  params = {'startAt':int(start_date.timestamp()), 'endAt':int(end_date.timestamp()), 'type':gran_to_string(granularity), 'symbol':product}
  response = connect(KUCOIN_CANDLES, params)
  response_text = response.text
  response_data = js.loads(response_text);
  if (response_data["code"] != "200000"):
    raise Exception("Illegal response: " + response_text)
  
  df_history = pd.DataFrame(response_data["data"])

  # kucoin is weird in that they don't have candles for everything. IF we don't have the requested
  # number of bars here, it throws off the whole algo. I don't want to try and project so we 
  # just won't trade those instruments
  got_bars = len(df_history)
  if ( got_bars < delta-1):
    raise Exception("Requested:" + str(delta) + " bars " + " but only got:" + str(got_bars))

  df_history.columns = ['time','open','close','high','low','volume', 'amount']
  df_history['time'] = [datetime.fromtimestamp(int(x)) for x in df_history['time']]
  df_history['open'] = [float(x) for x in df_history['open']]
  df_history['close'] = [float(x) for x in df_history['close']]
  df_history['high'] = [float(x) for x in df_history['high']]
  df_history['low'] = [float(x) for x in df_history['low']]
  df_history['low'] = [float(x) for x in df_history['low']]
  df_history['volume'] = [float(x) for x in df_history['volume']]
  df_history['amount'] = [float(x) for x in df_history['amount']]
  return df_history
  
def gran_to_string(granularity):
  #todo implement this actually
  return "1day"

def get_coin_data_frames(time, product, granularity='86400', feature_set = ["Close", "Volume", "Trend"]):
  if coin_base:
    df_raw = coinbase_json_to_df(time, product, granularity)
  else:
    df_raw = ku_coin_json_to_df(time, product, granularity)

  df_btc_history = df_raw
  if len(df_btc_history.index) == 0:
    print("No data for ", product)
  
  df_btc_history = df_btc_history.rename(columns={"time":"Date", "open":"Open", "high":"High", "low":"Low", "close":"Close", "volume":"Volume"})
  df_btc_history = sort_date(df_btc_history)
  df_btc_history = append_price_dif_(df_btc_history)
  df_btc_history = append_15d_slope(df_btc_history)
  df_btc_features = df_btc_history[feature_set]
  df_history_scaled = sc.fit_transform(df_btc_features)
  return [df_btc_history, df_btc_features, df_history_scaled, df_raw]

def build_profit_estimate(predicted, df_btc_history):
  df_predicted_chart = pd.DataFrame();
  df_predicted_chart["Date"] = df_btc_history["Date"]
  df_predicted_chart["Predicted"] = predicted
  df_predicted_chart["Predicted-Target"] = df_predicted_chart["Predicted"].shift(-1)
  df_predicted_chart["Predicted-Diff"] = df_predicted_chart["Predicted-Target"] - df_predicted_chart["Predicted"]
  df_predicted_chart["Should-Trade"] = np.where(df_predicted_chart["Predicted-Diff"] > 0, True, False)
  df_predicted_chart["RealDiff"] = df_btc_history["Diff"]
  df_predicted_chart["Percent"] = df_predicted_chart["RealDiff"] / df_btc_history["Close"]
  df_predicted_chart["Profit"] = np.where(df_predicted_chart["Should-Trade"] > 0, df_predicted_chart["Percent"] * budget, 0)
  profit = df_predicted_chart["Profit"].sum()
  return [df_predicted_chart, profit]

def debug_prediction_frame(predicted, df_history, df_history_scaled):
  df_predicted_chart = pd.DataFrame();
  df_predicted_chart["Date"] = df_history["Date"]
  df_predicted_chart["Predicted"] = predicted
  df_predicted_chart["Original"] = df_history_scaled[:,0]
  df_predicted_chart["Original-Target"] = df_history_scaled[:,2]
  df_predicted_chart["Target-Date"] = df_predicted_chart["Date"].shift(-1)
  df_predicted_chart["Predicted-Diff"] = df_predicted_chart["Predicted"] - df_predicted_chart["Original"]
  df_predicted_chart["Actual-Diff"] = df_predicted_chart["Original-Target"] - df_predicted_chart["Original"]
  df_predicted_chart["Should-Trade"] = np.where(df_predicted_chart["Predicted-Diff"] > 0, True, False)
  df_predicted_chart["Close"] = df_history["Close"]
  df_predicted_chart["Target"] = df_history["Target"]
  df_predicted_chart["RealDiff"] = df_history["Diff"]
  df_predicted_chart["Percent"] = df_predicted_chart["RealDiff"] / df_predicted_chart["Close"]
  df_predicted_chart["Profit"] = np.where(df_predicted_chart["Should-Trade"] > 0, df_predicted_chart["Percent"] * budget, 0)
  return df_predicted_chart

def get_all_products():
  if coin_base:
    return get_all_coinbase_products()
  
  if ku_coin:
    return get_all_kucoin_products()

def get_all_kucoin_products():
  response = connect(KUCOIN_PRODUCTS, {})
  products = js.loads(response.text)
  df_products = pd.DataFrame(products["data"]["ticker"])
  df_products = df_products.rename(columns={"symbol":"id"})
  return df_products

def get_all_coinbase_products():
  response = connect(COINBASE_PRODUCTS, {})
  response_text = response.text
  df_products = pd.read_json(response_text)
  return df_products

def predict_trade(model, product, bars, npa_scaled=[]):
  
  if len(npa_scaled) == 0:
    print("downloading...")
    [df_full, df_features, npa_scaled, df_raw] = get_coin_data_frames(bars, product)

  predicted = model.predict(npa_scaled).flatten()
  
  #convert to data frames that have the correct shape for being unscaled
  df_scaled = pd.DataFrame(npa_scaled, columns = ["Close", "Volume", "Trend"])
  
  # I want to believe that scaling happens on a per column basis, we only care about
  # price here so we will dummy out volume and trend and use the scaler on it
  # this kinda sucks, if we add features we'll need to add them here for unscaling
  df_temp = pd.DataFrame(predicted, columns = ["Close"])
  df_temp["Volume"] = 0
  df_temp["Trend"] = 0
  
  # unscale them both
  df_temp = pd.DataFrame(sc.inverse_transform(df_temp), columns = ["Close", "Volume", "Trend"])
  df_trade = pd.DataFrame(sc.inverse_transform(df_scaled), columns = ["Close", "Volume", "Trend"])
  
  # add predicted
  df_trade["Predicted"] = df_temp["Close"]
  df_trade = df_trade.tail(1)

  # add the product, derive a move and percent
  df_trade["Product"] = row.id;
  df_trade["Move"] = df_trade["Predicted"] - df_trade["Close"]
  df_trade["Percent"] = (df_trade["Move"] / df_trade["Close"]) * 100
  df_trade["RawPercent"] = df_trade["Move"] / df_trade["Close"]
  df_trade["250Fees"] = (250 * 0.004) * 2
  df_trade["5kFees"] = (5000 * 0.004) * 2
  df_trade["10kFees"] = (10000 * 0.0025) * 2
  df_trade["250Profit"] = (250 * df_trade["RawPercent"]) - df_trade["250Fees"] 
  df_trade["5kProfit"] = (5000 * df_trade["RawPercent"]) - df_trade["5kFees"]
  df_trade["10k0Profit"] = (10000 * df_trade["RawPercent"]) - df_trade["10kFees"]
  return df_trade

def get_yf_training_set_for(df):
  target_df = append_price_dif(df)
  target_df = append_15d_slope(target_df)
  features = target_df[["Open", "High", "Low", "Close", "Volume", "Trend", "Target"]]
  scaled_features = scale_data(features)
  return extract_training(scaled_features, len(target_df),len(features.columns)-1)

def extract_training(scaled_features, length, num_features):
  X = []
  y = []
  
  for i in range(0, length):
    X.append(scaled_features [i][0:num_features])
    y.append(scaled_features [i][num_features])
  X = np.asarray(X)
  y = np.asarray(y)
  return [scaled_features, X, y]

def get_training_set_for(ticker):
  target_df = get_single_stock(all_stocks_price_df, all_stocks_vol_df, ticker)
  target_df = append_price_dif(target_df)
  target_df = append_15d_slope(target_df)
  features = target_df[["Close", "Volume", "Trend", "Target"]]
  scaled_features = scale_data(features)
  return extract_training(scaled_features, len(target_df), len(features.columns)-1)
  
def train_model(model, X, y):

  # One day we might need test, but for now we don't we can use another
  # time series, we have so many
  # Split the data
  #split = int(0.7 * len(X))
  #X_train = X[:split]
  #y_train = y[:split]
  #X_test = X[split:]
  #y_test = y[split:]

  # Reshape the 1D arrays to 3D arrays to feed in the model
  X_train = np.reshape(X, (X.shape[0], X.shape[1], 1))
  
  # Create an early stopping callback
  early_stopping = EarlyStopping(monitor='val_loss', patience=5)
  
  history = model.fit(
      X_train, y,
      epochs = 20,
      batch_size = 32,
      validation_split = 0.2,
      callbacks=[early_stopping]
  )
  return [model, history]

#pull training data 
all_stocks_price_df = sort_date(pd.read_csv('/content/drive/My Drive/Colab Notebooks/stock.csv'))
all_stocks_vol_df = sort_date(pd.read_csv("/content/drive/My Drive/Colab Notebooks/stock_volume.csv"))
spy_df = sort_date(pd.read_csv('/content/drive/My Drive/SPY.csv'))
cat_df = sort_date(pd.read_csv('/content/drive/My Drive/CAT.csv'))
f_df = sort_date(pd.read_csv('/content/drive/My Drive/F.csv'))
xom_df = sort_date(pd.read_csv('/content/drive/My Drive/XOM.csv'))
ibm_df = sort_date(pd.read_csv('/content/drive/My Drive/IBM.csv'))

In [ ]:
def prepare_yahoo_df(df):
  # Preprocess the data
  # Normalize the features
  # Convert the data to a supervised learning problem
  X = df.drop(columns=['Close', 'Date', 'Adj Close'])
  y = df['Close']
  return [X, y]

array([[1.43389322e-03, 1.00240862e-03, 2.16389017e-03, 1.14578166e-03],
       [1.43389322e-03, 1.64681416e-03, 2.66879787e-03, 5.45681385e-04],
       [2.00745050e-03, 1.93321662e-03, 3.02944623e-03, 2.25138059e-04],
       ...,
       [7.79363952e-01, 7.85152284e-01, 7.66207516e-01, 1.14940614e-01],
       [7.71563587e-01, 7.77934956e-01, 7.73732106e-01, 6.86044230e-02],
       [7.78767500e-01, 7.78141157e-01, 7.77471790e-01, 5.37999091e-02]])

# Get or Train a Model

In [ ]:
if 1:
  #load models
  print("loading models from disk")
  model_orig = keras.models.load_model(model_path + "/model_orig.h15")
  model_ohlc = keras.models.load_model(model_path + "/model_ohlc.h15")
  model_att1 = keras.models.load_model(model_path + "/model_att1.h15")
  model_att2 = keras.models.load_model(model_path + "/model_att2.h15")
else:
  #if file_exists:
  #  print("hello")
  #  model = keras.models.load_model(model_path)
  #else:

  [scaled_features, X, y] = get_training_set_for("sp500")  
  [scaled_features1, X1, y1] = get_yf_training_set_for(spy_df)  

  model_orig = build_model(num_features, 1)
  model_ohlc = build_model(6, 1)
  model_att1 = build_attention_model(num_features, 1)
  model_att2 = build_attention_model(6, 1)

  [model_att1, history] = train_model(model_att1, X, y)
  [model_orig, history] = train_model(model_orig, X, y)
  [model_ohlc, history] = train_model(model_ohlc, X1, y1)
  [model_att2, history] = train_model(model_att2, X1, y1)



In [37]:
if 1:
  [scaled_features, X, y] = get_training_set_for("IBM")  
  [model_orig, history] = train_model(model_orig, X, y)
  [scaled_features, X, y] = get_training_set_for("T")  
  [model_orig, history] = train_model(model_orig, X, y)
  [scaled_features, X, y] = get_training_set_for("BA")  
  [model_orig, history] = train_model(model_orig, X, y)
  [scaled_features, X, y] = get_training_set_for("TSLA")  
  [model_orig, history] = train_model(model_orig, X, y)


Epoch 1/20
54/54 [==============================] - 2s 42ms/step - loss: 0.0011 - val_loss: 6.2236e-04
Epoch 2/20
54/54 [==============================] - 2s 36ms/step - loss: 0.0014 - val_loss: 6.6088e-04
Epoch 3/20
54/54 [==============================] - 2s 37ms/step - loss: 0.0011 - val_loss: 8.3525e-04
Epoch 4/20
54/54 [==============================] - 2s 35ms/step - loss: 9.0291e-04 - val_loss: 6.6132e-04
Epoch 5/20
54/54 [==============================] - 2s 31ms/step - loss: 6.5775e-04 - val_loss: 5.9676e-04
Epoch 6/20
54/54 [==============================] - 3s 60ms/step - loss: 9.0454e-04 - val_loss: 0.0013
Epoch 7/20
54/54 [==============================] - 2s 46ms/step - loss: 7.3869e-04 - val_loss: 6.7797e-04
Epoch 8/20
54/54 [==============================] - 2s 32ms/step - loss: 6.7201e-04 - val_loss: 5.5436e-04
Epoch 9/20
54/54 [==============================] - 2s 32ms/step - loss: 5.7627e-04 - val_loss: 5.5499e-04
Epoch 10/20
54/54 [==============================] - 

In [ ]:
# additional training?
if 1:

  [scaled_features1, X1, y1] = get_yf_training_set_for(cat_df)  
  [model_ohlc, history] = train_model(model_ohlc, X1, y1)
  [scaled_features1, X1, y1] = get_yf_training_set_for(f_df)  
  [model_ohlc, history] = train_model(model_ohlc, X1, y1)
  [scaled_features1, X1, y1] = get_yf_training_set_for(ibm_df)  
  [model_ohlc, history] = train_model(model_ohlc, X1, y1)
  [scaled_features1, X1, y1] = get_yf_training_set_for(xom_df)  
  [model_ohlc, history] = train_model(model_ohlc, X1, y1)

# Visualize and Backtest

In [ ]:

# Run Random Search flat, attention flat, random ohlc and flat ohlc against these two models
# Loss isn't cutting it, its always 0?
# After we find the winner above, run it trained on spy vs trained on all

[btc_history, df_btc_features, df_history_scaled, df_raw] = get_coin_data_frames(180, "FCON-USDT")
[btc_history1, df_btc_features1, df_history_scaled1, df_raw1] = get_coin_data_frames(180, "FCON-USDT", 86400, ["Open", "High", "Low", "Close", "Volume", "Trend"])

loss1 = model_orig.evaluate(df_history_scaled)
loss2 = model_ohlc.evaluate(df_history_scaled1)
loss3 = model_att1.evaluate(df_history_scaled)
loss4 = model_att2.evaluate(df_history_scaled1)

#print("Losses: ", loss1, loss2, loss3, loss4)
print("Losses: ", loss1, loss2, loss3, loss4)

budget = 3000

predicted_orig = model_orig.predict(df_history_scaled).flatten()
predicted_ohlc = model_ohlc.predict(df_history_scaled1).flatten()
predicted_att1 = model_att1.predict(df_history_scaled).flatten()
predicted_att2 = model_att2.predict(df_history_scaled1).flatten()


[df_profit, profit1] = build_profit_estimate(predicted_orig, btc_history)
[df_profit, profit2] = build_profit_estimate(predicted_ohlc, btc_history1)
[df_profit, profit3] = build_profit_estimate(predicted_att1, btc_history)
[df_profit, profit4] = build_profit_estimate(predicted_att2, btc_history1)


df_chart = debug_prediction_frame(predicted_orig, btc_history, df_history_scaled)
df_chart["Predicted-ohlc"] = predicted_ohlc
df_chart["Predicted-att1"] = predicted_att1
df_chart["Predicted-att2"] = predicted_att2

#interactive_plot(df_chart[["Date","Original", "Predicted-ohlc", "Predicted", "Predicted-atten", "Predicted-opt"]], "Wtf")
interactive_plot(df_chart[["Date","Original", "Predicted-ohlc", "Predicted", "Predicted-att1", "Predicted-att2"]], "Wtf")
print("Profits:", profit1, profit2, profit3, profit4)


In [ ]:
# Save modesl
model_orig.save(model_path + "/model_orig.h15")
model_ohlc.save(model_path + "/model_ohlc.h15")
model_att1.save(model_path + "/model_att1.h15")
model_att2.save(model_path + "/model_att2.h15")

# What has a buy indicator for tomorrow?

In [63]:
# Fetch the top 10 and see if they predict up
df_products = get_all_products()
df_products = df_products[df_products.id.str.endswith('USDT')]

if coin_base:
  df_products = df_products[df_products.trading_disabled == False]
  df_products = df_products[df_products.cancel_only == False]

df_trades = pd.DataFrame();
bars = 91
counter = 0;
for index, row in df_products.iterrows():
  try:
    print("fetching: ", row.id)
    [df_full, df_features, npa_scaled, df_raw] = get_coin_data_frames(bars, row.id)
    df_trade = predict_trade(model_orig, row.id, bars, npa_scaled)
    df_trade_ohlc = predict_trade(model_ohlc, row.id, bars, npa_scaled)
    df_trade_att1 = predict_trade(model_att1, row.id, bars, npa_scaled)
    df_trade_att2 = predict_trade(model_att2, row.id, bars, npa_scaled)
    
    # we need to unscale the predicted values so that we have an entry and exit point
    # entry should be roughly close and exit should be roughly predicted

    # Stick this on the end of the main dataframe
    df_trade["prd-ohlc"] = df_trade_ohlc["Predicted"]
    df_trade["pct-ohlc"] = df_trade_ohlc["Percent"]
    df_trade["prd-att1"] = df_trade_att1["Predicted"]
    df_trade["pct-att1"] = df_trade_att1["Percent"]
    df_trade["prd-att2"] = df_trade_att2["Predicted"]
    df_trade["pct-att2"] = df_trade_att2["Percent"]
    df_trades = df_trades.append(df_trade);
    
    #counter+=1
    #if counter > 5:
    #  break
  except Exception as inst:
    #raise inst
    print("Error: ", inst)
  time.sleep(1)
df_trades.reset_index()
df_buys = df_trades[df_trades['Move'] > 0] 
df_shorts = df_trades[df_trades['Move'] < 0] 




fetching:  NKN-USDT
3/3 [==============================] - 0s 22ms/step
fetching:  GEM-USDT
Error:  Requested:91 bars  but only got:83
fetching:  CUSD-USDT
3/3 [==============================] - 0s 9ms/step
fetching:  LTC3L-USDT
3/3 [==============================] - 0s 11ms/step
fetching:  OAS-USDT
Error:  Requested:91 bars  but only got:21
fetching:  KNC-USDT
3/3 [==============================] - 0s 10ms/step
fetching:  LYM-USDT
3/3 [==============================] - 0s 12ms/step
fetching:  HAI-USDT
3/3 [==============================] - 0s 11ms/step
fetching:  MITX-USDT
3/3 [==============================] - 0s 18ms/step
fetching:  PDEX-USDT
3/3 [==============================] - 0s 10ms/step
fetching:  FLAME-USDT
3/3 [==============================] - 0s 9ms/step
fetching:  EPX-USDT
3/3 [==============================] - 0s 10ms/step
fetching:  AOG-USDT
3/3 [==============================] - 0s 10ms/step
fetching:  ATOM3L-USDT
3/3 [==============================] - 0s 11ms/step
fe

In [66]:
df_buys


,Close,Volume,Trend,Predicted,Product,Move,Percent,RawPercent,250Fees,5kFees,10kFees,250Profit,5kProfit,10k0Profit,prd-ohlc,pct-ohlc,prd-att1,pct-att1,prd-att2,pct-att2
75,0.00151,79.20900,0.00001,0.00154,LTC3L-USDT,0.00002,1.56584,0.01566,2.00000,40.00000,50.00000,1.91460,38.29203,106.58405,0.00105,-30.32345,0.00143,-5.64791,0.00102,-32.82380
74,0.45700,7173.68130,-0.00567,0.45818,KNC-USDT,0.00118,0.25897,0.00259,2.00000,40.00000,50.00000,-1.35258,-27.05168,-24.10336,0.49523,8.36502,0.43407,-5.01841,0.50358,10.19332
75,0.00283,3203.51380,-0.00003,0.00284,MITX-USDT,0.00001,0.50140,0.00501,2.00000,40.00000,50.00000,-0.74650,-14.92992,0.14015,0.00306,8.34176,0.00268,-5.25037,0.00312,10.23932
75,1.47400,308.87730,0.00070,1.48820,PDEX-USDT,0.01420,0.96341,0.00963,2.00000,40.00000,50.00000,0.40853,8.17066,46.34133,1.27010,-13.83280,1.44340,-2.07599,1.25144,-15.09889
74,0.02065,8261181.22040,-0.00011,0.02097,FLAME-USDT,0.00032,1.55088,0.01551,2.00000,40.00000,50.00000,1.87720,37.54395,105.08789,0.02199,6.47101,0.02062,-0.15960,0.02210,7.03355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.00578,4064.35950,0.00004,0.00578,CERE-USDT,0.00000,0.04342,0.00043,2.00000,40.00000,50.00000,-1.89145,-37.82904,-45.65808,0.00468,-18.97032,0.00569,-1.48555,0.00455,-21.22899
75,0.00851,1183798.32480,0.00022,0.00869,UPO-USDT,0.00018,2.15975,0.02160,2.00000,40.00000,50.00000,3.39938,67.98767,165.97535,0.00538,-36.75272,0.00823,-3.32711,0.00504,-40.76633
75,0.00144,634187.17750,-0.00002,0.00145,2CRZ-USDT,0.00000,0.27325,0.00273,2.00000,40.00000,50.00000,-1.31688,-26.33755,-22.67511,0.00158,9.69361,0.00134,-7.16794,0.00162,12.08027
75,0.40430,2397.85070,-0.00295,0.40686,RNDR-USDT,0.00256,0.63264,0.00633,2.00000,40.00000,50.00000,-0.41841,-8.36815,13.26370,0.44262,9.47885,0.37879,-6.30910,0.45163,11.70758


In [ ]:
df_trade = predict_trade(model_orig, "REN-USD", 90)
df_trade

In [ ]:
with open(data_path, 'w', encoding = 'utf-8-sig') as f:
  df_trades.to_csv(f)

In [ ]:
df_shorts[df_shorts["Product"] == "MPL-USD"]